# Uber Movement Competition

In [1]:
import pandas as pd
# import pandas_profiling as pf 
import os
import numpy as np
import seaborn as sns
import matplotlib as plt
import datetime
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
#from tqdm import tqdm
#tqdm.pandas()
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.neighbors import NearestNeighbors
%matplotlib inline
# !pip install pandas_profiling

import gc
import catboost
from catboost import CatBoostClassifier,Pool,cv

### Loading the Train Dataset

In [2]:
train = pd.read_csv('Data/train.csv')

### Handle Bad Data

In [3]:
train=train.rename(columns = {'Occurrence Local Date Time':'Datetime'})
train=train.rename(columns = {'road_segment_id':'segment_id'})

### Date conversion (make dayfirst=True)
train['Datetime'] = pd.to_datetime(train['Datetime'], dayfirst=True)

In [4]:
bad_seg = ['-34.0436786939','-33.9622761744','-33.8891283413','-34.0894652753','-33.9680008638']
for seg in bad_seg:
    train.loc[train['segment_id'] == seg , 'longitude']  = train.loc[train['segment_id'] == seg , 'latitude']
    train.loc[train['segment_id'] == seg , 'latitude']  = train.loc[train['segment_id'] == seg , 'segment_id']
    train.loc[train['segment_id'] == seg , 'segment_id']  = np.nan

In [5]:
train[train.segment_id.isna()]

,EventId,Datetime,Reporting Agency,Cause,Subcause,Status,longitude,latitude,segment_id
16693,88632,2016-12-09 04:45:00,NaN,Cam,Accident,Single Vehicle,18.5642,-33.9622761744,NaN
23455,100642,2017-05-16 07:22:00,NaN,cdam,Congestion,Any,18.8397,-34.0894652753,NaN
23747,101150,2017-05-21 08:30:00,NaN,cam,Routine Road Maintenance,Any,18.5798,-33.9680008638,NaN
27586,108150,2017-08-30 07:12:00,NaN,cam,Congestion,Any,18.6146,-33.8891283413,NaN
30341,112789,2017-10-21 08:03:00,NaN,cam,Stationary Vehicle,Vehicle On Shoulder,18.7424,-34.0436786939,NaN


In [6]:
train.loc[train['EventId'] == 88632, 'segment_id']   = '6OTRJQF'
train.loc[train['EventId'] == 100642, 'segment_id']   = 'Q03FQ74'
train.loc[train['EventId'] == 101150, 'segment_id']   = 'S2QPOTD'
train.loc[train['EventId'] == 108150, 'segment_id']   = 'LNO3W8J'
train.loc[train['EventId'] == 112789, 'segment_id']   = 'IUTMY1U'

In [7]:
train[train.segment_id.isna()]

,EventId,Datetime,Reporting Agency,Cause,Subcause,Status,longitude,latitude,segment_id


### Accident Features

In [8]:
Accident = train.segment_id.value_counts().reset_index().rename(columns = {'index':'Road_Segment',
                                                                    'segment_id':'Accident'})
Accident['Accident'] = Accident['Accident'].astype('int32')

In [9]:
ls Data

Accident_Per_SegmentId.csv  seg_Length.csv            seg_pavement_v2.csv
Add_feat_cause.csv          seg_Length_v2.csv         seg_roadtype.csv
GeoData_001to1107.csv       SegmentId_and_suburb.csv  seg_roadtype_v2.csv
GeoData_1107tolast.csv      segment_id_suburb_v2.csv  train.csv
road_segments/              seg_pavement.csv          weather.csv


In [10]:
# test = pd.read_csv('Data/SampleSubmission.csv',error_bad_lines=False)
# Suburb = pd.read_csv("Data/SegmentId_and_suburb.csv")

Suburb = pd.read_csv("Data/segment_id_suburb_v2.csv")
road_type = pd.read_csv("Data/seg_roadtype_v2.csv")
road_pavement = pd.read_csv("Data/seg_pavement_v2.csv")
road_Length = pd.read_csv("Data/seg_Length_v2.csv")

In [11]:
train.head()

,EventId,Datetime,Reporting Agency,Cause,Subcause,Status,longitude,latitude,segment_id
0,60558,2016-01-01 00:53:00,Cam,Stationary Vehicle,Vehicle On Shoulder,Closed,18.5408955032,-33.8883,S0B3CGQ
1,60559,2016-01-01 00:54:00,CAMERA,Accident,With A Fixed Object,Closed,18.9307563219,-34.1409,RYJYAPI
2,60560,2016-01-01 02:26:00,Law Enforcement,Accident,Multi Vehicle,Closed,18.5533575029,-33.9592,U3KP57C
3,60561,2016-01-01 02:56:00,CAMERA,Stationary Vehicle,Vehicle On Shoulder,Closed,18.6775561589,-33.8953,RY0TRQ8
4,60562,2016-01-01 03:40:00,CAMERA,Accident,Multi Vehicle,Closed,18.8371319682,-34.0871,8LOVJZ3


## Weather Data

In [12]:
w = pd.read_csv('Data/weather.csv', sep=";", skiprows=6, usecols=range(14),
               parse_dates=['Local time in Cape Town (airport)'])

In [13]:
w.head()

,Local time in Cape Town (airport),T,P0,P,U,DD,Ff,ff10,WW,W'W',c,VV,Td,Unnamed: 13
0,2019-04-04 23:00:00,15.0,762.0,765.8,77.0,Wind blowing from the south-southeast,6.0,NaN,NaN,NaN,"Scattered clouds (40-50%) 720 m, broken clouds...",10.0 and more,11.0,NaN
1,2019-04-04 22:00:00,16.0,761.2,765.0,77.0,Wind blowing from the south-southeast,6.0,NaN,NaN,NaN,"Scattered clouds (40-50%) 570 m, broken clouds...",10.0 and more,12.0,NaN
2,2019-04-04 21:59:00,16.0,761.2,765.0,77.0,Wind blowing from the south-southeast,6.0,NaN,NaN,NaN,"Scattered clouds (40-50%) 570 m, broken clouds...",10.0 and more,12.0,NaN
3,2019-04-04 21:01:00,15.0,761.2,765.0,94.0,Wind blowing from the south-southeast,6.0,NaN,NaN,NaN,"Scattered clouds (40-50%) 150 m, broken clouds...",10.0 and more,14.0,NaN
4,2019-04-04 21:00:00,15.0,761.2,765.0,94.0,Wind blowing from the south-southeast,6.0,NaN,NaN,NaN,"Scattered clouds (40-50%) 150 m, broken clouds...",10.0 and more,14.0,NaN


In [14]:
w['dt'] = w['Local time in Cape Town (airport)'].dt.round('H')
w['dt_2'] = w['Local time in Cape Town (airport)'].dt.floor('H')

w_cols = ['dt','dt_2', 'T', 'P0', 'P', 'U', 'Ff']

### Get Extra Feature from Segment

In [15]:
### Count of occurance of incident
Seg_Group = train.groupby(['segment_id','Datetime']).size().reset_index(name='Counts')

### Mapping the Segment_id - from text to number
LE = LabelEncoder()
Seg_Group['segmentid_num'] = LE.fit_transform(Seg_Group['segment_id'])

# Make all 3 digits of segments
Seg_Group['segmentid_num3'] = [str(x).zfill(3) for x in Seg_Group['segmentid_num']]

# Add 'seg_' to 3digit segment_id
Seg_Group['segment_idx'] = 'seg_'+Seg_Group.segmentid_num3.astype('str')

In [16]:
Seg_Group.head()

,segment_id,Datetime,Counts,segmentid_num,segmentid_num3,segment_idx
0,03RHJ3G,2016-01-01 09:02:00,1,0,000,seg_000
1,03RHJ3G,2016-01-04 09:46:00,1,0,000,seg_000
2,03RHJ3G,2016-01-04 13:29:00,1,0,000,seg_000
3,03RHJ3G,2016-01-05 14:41:00,1,0,000,seg_000
4,03RHJ3G,2016-01-05 16:10:00,1,0,000,seg_000


### Extract unique Segment Id

In [17]:
## Drop duplicate segment_id
SegIdQ = Seg_Group[['segment_id','segment_idx','segmentid_num']].drop_duplicates()
# segment_id_1
SegIdQx = SegIdQ['segment_id'].unique()
# segmen_id_2
SegIdQx2 = SegIdQ['segment_idx'].unique()

### Add centre of Segment

In [18]:
train.longitude = train.longitude.astype(float)
train.latitude = train.latitude.astype(float)

## Get centre location of the segment
location =  train.groupby('segment_id').median()[['longitude', 'latitude']]
SegIdQ = SegIdQ.merge(location, left_on=['segment_id'], how="left", right_index=True)

In [19]:
SegIdQ.head()

,segment_id,segment_idx,segmentid_num,longitude,latitude
0,03RHJ3G,seg_000,0,18.459897,-33.943257
582,044ZYVD,seg_001,1,18.554505,-33.890213
613,086LLYS,seg_002,2,18.432873,-33.918816
637,0B0QOEN,seg_003,3,18.477737,-33.917932
790,0F3OY57,seg_004,4,18.773261,-34.057950


### Add other segment related feat

In [20]:
## Add centre Accident_count to data_set
SegIdQ = pd.merge(SegIdQ, Accident[['Road_Segment','Accident']], left_on='segment_id', right_on='Road_Segment', how='left') # Segment locations
SegIdQ = pd.merge(SegIdQ, Suburb[['segment_id','suburb']], left_on='segment_id', right_on='segment_id', how='left') # Segment locations
# SegIdQ = pd.merge(SegIdQ, Cause, left_on='segment_id', right_on='Road_Segment', how='left') # Segment locations

In [21]:
## Add centre Other  to data_set
SegIdQ = pd.merge(SegIdQ, road_Length, left_on='segment_id', right_on='segment_id', how='left') # Segment locations
SegIdQ = pd.merge(SegIdQ, road_type, left_on='segment_id', right_on='segment_id', how='left') # Segment locations
SegIdQ = pd.merge(SegIdQ, road_pavement, left_on='segment_id', right_on='segment_id', how='left') # Segment locations

### Merging Train Dataset with Segment Features from NoteBook(`Geo-Spatial Features Extraction_v3`)

In [22]:
trainX = pd.merge(train, SegIdQ, on=['segment_id'], how="left")
print('Train # B4 merging',len(train))
print('Train # After merging',len(trainX))

Train # B4 merging 53845
Train # After merging 53845


In [23]:
# del SegIdQ
del train

In [24]:
# trainX.to_csv('train_transformed.csv', index=False)

### CreateTrain Target

In [25]:
## Get day not in Train day and save as missing_days
Main_day = pd.date_range('2016-01-01','2018-12-31',
                           freq="1D")
Main_day = pd.DataFrame({'datetime':Main_day.date})

train_days = trainX['Datetime'].dt.date.drop_duplicates()
missing_days = Main_day[~Main_day['datetime'].isin(train_days)]

del train_days, Main_day
missing_days['datetime'] = pd.to_datetime(missing_days.datetime)

In [26]:
Data1Hour = pd.date_range('2016-01-01',
                          '2018-12-31 23:00:00',
                           freq="1h")
Data1Hour = pd.DataFrame({'datetime':Data1Hour})
###Remove missing date from date range
Data1Hour = Data1Hour[~Data1Hour['datetime'].dt.date.isin(missing_days['datetime'].dt.date)]

# Generate Target
for SegId in SegIdQx2:
    Data1Hour[str(SegId)] = 0
    events = trainX.loc[trainX['segment_idx'] == SegId]
    dts = events['Datetime'].dt.floor('H')
    dates = dts.astype(str).unique()
    Data1Hour.loc[Data1Hour['datetime'].isin(dates), SegId] = 1


In [27]:
Data1Hour.shape

(24672, 545)

### Create Test Target

In [28]:
test1Hour = pd.date_range('2019-01-01 01:00:00',
                          '2019-03-31 23:00:00' ,
                           freq="1h")

test1Hour = pd.DataFrame({'datetime':test1Hour})

# Generate Target
for SegId in SegIdQx2:
    test1Hour[str(SegId)] = np.NAN


In [29]:
test1Hour.set_index('datetime', inplace=True)
Data1Hour.set_index('datetime', inplace=True)

### Convert seg to Id

In [30]:
Data1Hour = Data1Hour.reset_index()
test1Hour = test1Hour.reset_index()

In [31]:
Data1Hour = pd.melt(Data1Hour, id_vars=['datetime'])
Data1Hour.rename(columns={'variable': 'seg', 'value': 'target'}, inplace=True)
Data1Hour['seg'] = Data1Hour.seg.map(lambda x: int(x[4:]))
Data1Hour.set_index('datetime', inplace=True)
# Data1Hour.sort_values('datetime', inplace=True)
Data1Hour = Data1Hour.sort_values('datetime')



test1Hour = pd.melt(test1Hour, id_vars=['datetime'])
test1Hour.rename(columns={'variable': 'seg', 'value': 'target'}, inplace=True)
test1Hour['seg'] = test1Hour.seg.map(lambda x: int(x[4:]))
test1Hour.set_index('datetime', inplace=True)

test1Hour = test1Hour.sort_values('datetime')

In [32]:
Data1Hour.head()

,seg,target
datetime,,
2016-01-01,0,0
2016-01-01,170,0
2016-01-01,171,0
2016-01-01,172,0
2016-01-01,173,0


### Timebased FeatEngineering

In [33]:
attr = ["Hour","Year", 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear','quarter',"weekday_name",
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']

for n in attr: 
    Data1Hour[n] = getattr(Data1Hour.index, n.lower())
    test1Hour[n] = getattr(test1Hour.index, n.lower())

# Data1Hour['Elapsed'] = Data1Hour.index.astype(np.int64) // 10 ** 9
# test1Hour['Elapsed'] = test1Hour.index.astype(np.int64) // 10 ** 9

In [34]:
## WeekEnd
Data1Hour['IsWeekend'] = np.where(Data1Hour['Dayofweek'] < 5,0,1)
test1Hour['IsWeekend'] = np.where(test1Hour['Dayofweek'] < 5,0,1)

In [35]:
# Holiday 
from pandas.tseries.holiday import *
from pandas.tseries.offsets import CustomBusinessDay

class SouthAfricaHoliday(AbstractHolidayCalendar):
   rules = [
     Holiday('New Year', month=1, day=1, observance=sunday_to_monday),
     Holiday('Human Rights Day', month=3, day=21, observance=sunday_to_monday),
     Holiday('Good Friday', month=1, day=1, offset=[Easter(), Day(-2)]),
     Holiday('Easter Monday', month=1, day=1, offset=[Easter(), Day(1)]),
     Holiday('Freedom Day', month=4, day=27, observance=sunday_to_monday),
     Holiday('Workers Day', month=5, day=1,observance=sunday_to_monday),
     Holiday('Youth Day', month=6, day=16, observance=sunday_to_monday),
     Holiday('Womens Day', month=8, day=9, observance=sunday_to_monday),
     Holiday('Heritage Day', month=9, day=24, observance=sunday_to_monday),
     Holiday('Day of Reconcilation', month=12, day=16, observance=sunday_to_monday),
     Holiday('Goodwill Day', month=12, day=26, observance=sunday_to_monday),
     Holiday('Christmas', month=12, day=25)
   ]

In [36]:
Data1Hour.index.min(), test1Hour.index.max()

(Timestamp('2016-01-01 00:00:00'), Timestamp('2019-03-31 23:00:00'))

In [37]:
hol= SouthAfricaHoliday()
myholidays =hol.holidays(start=Data1Hour.index.min(), end =test1Hour.index.max())
hol_day = pd.Series(myholidays)

### election
election_day = "2019-08-03"
hol_day = hol_day.append(pd.to_datetime(pd.Series([election_day])),ignore_index=True)

In [38]:
## Create instance of holiday in a new column
Data1Hour['IsHoliday'] = np.where((pd.to_datetime(Data1Hour.index.date)).isin(hol_day) ,1,0)
test1Hour['IsHoliday'] = np.where((pd.to_datetime(test1Hour.index.date)).isin(hol_day) ,1,0)

In [39]:
# pay_day 

Data1Hour['PayDay'] = np.where((Data1Hour['Day'] < 25) ,0,1)
Data1Hour['PayDay_Teachers'] = np.where(Data1Hour['Day'] < 22,0,1)


test1Hour['PayDay'] = np.where((test1Hour['Day'] < 25) ,0,1)
test1Hour['PayDay_Teachers'] = np.where(test1Hour['Day'] < 22,0,1)

In [40]:
Data1Hour['min'] = Data1Hour.index.hour*60+Data1Hour.index.minute
test1Hour['min'] = test1Hour.index.hour*60+test1Hour.index.minute

Data1Hour['Hour_sin'] = np.sin(2 * np.pi * Data1Hour['Hour']/23.0)
Data1Hour['Hour_cos'] = np.cos(2 * np.pi * Data1Hour['Hour']/23.0)

test1Hour['Hour_sin'] = np.sin(2 * np.pi * test1Hour['Hour']/23.0)
test1Hour['Hour_cos'] = np.cos(2 * np.pi * test1Hour['Hour']/23.0)

In [41]:
Data1Hour.columns

Index(['seg', 'target', 'Hour', 'Year', 'Month', 'Week', 'Day', 'Dayofweek',
       'Dayofyear', 'quarter', 'weekday_name', 'Is_month_end',
       'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end',
       'Is_year_start', 'IsWeekend', 'IsHoliday', 'PayDay', 'PayDay_Teachers',
       'min', 'Hour_sin', 'Hour_cos'],
      dtype='object')

In [42]:
Data1Hour.shape

(13421568, 24)

In [43]:
Data1Hour.target.value_counts(True)

0    0.99617
1    0.00383
Name: target, dtype: float64

In [44]:
gc.collect()

163

### Preprocessing 2

In [45]:
## Memory reduce Function
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


In [46]:
train = reduce_mem_usage(Data1Hour)
test = reduce_mem_usage(test1Hour)

Memory usage of dataframe is 2062.37 MB
Memory usage after optimization is: 603.19 MB
Decreased by 70.8%
Memory usage of dataframe is 216.97 MB
Memory usage after optimization is: 96.00 MB
Decreased by 55.8%


In [47]:
train.target.value_counts(True)

0    0.99617
1    0.00383
Name: target, dtype: float64

In [48]:
train = train.sort_values('datetime')
test = test.sort_values('datetime')

## Merge New features to Train and Test DataSet

In [49]:
test.columns

Index(['seg', 'target', 'Hour', 'Year', 'Month', 'Week', 'Day', 'Dayofweek',
       'Dayofyear', 'quarter', 'weekday_name', 'Is_month_end',
       'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end',
       'Is_year_start', 'IsWeekend', 'IsHoliday', 'PayDay', 'PayDay_Teachers',
       'min', 'Hour_sin', 'Hour_cos'],
      dtype='object')

In [50]:
test['datetime'] = test.index
train['datetime'] = train.index

In [51]:
## Merge Weather
w = w[w_cols].drop_duplicates(['dt_2'])
train = pd.merge(train, w[w_cols], left_index=True, right_on='dt_2', how='left') # Weather
test = pd.merge(test, w[w_cols], left_index=True, right_on='dt_2', how='left') # Weather

In [52]:
## Merge Other New feature Stored in SegIdQ
train = pd.merge(train, SegIdQ, left_on=['seg'], right_on=['segmentid_num'], how="left")
test = pd.merge(test, SegIdQ, left_on=['seg'], right_on=['segmentid_num'], how="left")

In [53]:
test.set_index('datetime', inplace=True)
train.set_index('datetime', inplace=True)

In [54]:
test['datetime x segment_id'] = test.index.astype('str') + ' x ' + test['segment_id']

In [55]:
train.shape, test.shape

((13421568, 42), (1174496, 43))

## Modelling

In [56]:
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import confusion_matrix, f1_score, precision_recall_curve

In [57]:
train.columns

Index(['seg', 'target', 'Hour', 'Year', 'Month', 'Week', 'Day', 'Dayofweek',
       'Dayofyear', 'quarter', 'weekday_name', 'Is_month_end',
       'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end',
       'Is_year_start', 'IsWeekend', 'IsHoliday', 'PayDay', 'PayDay_Teachers',
       'min', 'Hour_sin', 'Hour_cos', 'dt', 'dt_2', 'T', 'P0', 'P', 'U', 'Ff',
       'segment_id', 'segment_idx', 'segmentid_num', 'longitude', 'latitude',
       'Road_Segment', 'Accident', 'suburb', 'length_1', 'roadno', 'surftype'],
      dtype='object')

In [58]:
### Columns to be deleted
del_col = [
       'Day', 
#          'Dayofweek', 'Hour', 
#          'Hour_cos', 'Hour_sin',
#        'IsHoliday', 'IsWeekend', 
       'Is_month_end', 'Is_month_start',
       'Is_quarter_end', 'Is_quarter_start', 
       'Is_year_end', 'Is_year_start',
       'PayDay', 
       'PayDay_Teachers', 
       'Week', 
        'min',
       'quarter', 
#        'seg',
    'dt',
     'dt_2',
     'T',
     'P0',
     'P',
     'U',
     'Ff',
#     'RoadNo',
#     'SURFTYPE'
      'Cause', 'Freq', 'CatNo_cause',
#     'longitude', 'latitude',
#     'Accident', 'suburb',
      'weekday_name','Dayofyear',
      'Road_Segment_x','segment_id','Road_Segment_y','Month', 'segmentid_num','segment_idx',       
      'target','Year','SegmentId','SegmentId_x','SegmentId_y','Road_Segment'
   ]

In [59]:
train.columns

Index(['seg', 'target', 'Hour', 'Year', 'Month', 'Week', 'Day', 'Dayofweek',
       'Dayofyear', 'quarter', 'weekday_name', 'Is_month_end',
       'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end',
       'Is_year_start', 'IsWeekend', 'IsHoliday', 'PayDay', 'PayDay_Teachers',
       'min', 'Hour_sin', 'Hour_cos', 'dt', 'dt_2', 'T', 'P0', 'P', 'U', 'Ff',
       'segment_id', 'segment_idx', 'segmentid_num', 'longitude', 'latitude',
       'Road_Segment', 'Accident', 'suburb', 'length_1', 'roadno', 'surftype'],
      dtype='object')

#### Specify Cat col And Encode Them

In [60]:
cat_cols = ['Hour','Dayofweek','seg','suburb','roadno','surftype']
LE = LabelEncoder()
for col in cat_cols:
    train[col] = train[col].replace({np.nan:'NaN'})
    train[col] = LE.fit_transform(train[col])
    
    test[col] = test[col].replace({np.nan:'NaN'})
    test[col] =  LE.transform(test[col])    

### Specify TrainSet
Train with only 2018 because it is the most recent  data

In [61]:
start      = '2018-01-01'
split_date = '2018-10-01'


val_train = train[(train.index >= start) & (train.index <='2018-12-31 23:00:00')]
val_test = train[train.index >= split_date]

val_train.shape, val_test.shape

((4752384, 42), (1201152, 42))

In [62]:
# Remove August because it contains very few number of records
val_train = val_train[~((val_train.index >= '2018-08-01') & (val_train.index <'2018-09-01'))]

In [63]:
print ('Val_train Target value count \n', val_train['target'].value_counts(True))
print ('Val_test Target value count \n', val_test['target'].value_counts(True))

Val_train Target value count 
 0    0.995385
1    0.004615
Name: target, dtype: float64
Val_test Target value count 
 0    0.993427
1    0.006573
Name: target, dtype: float64


### Features for training

In [64]:
features = [x for x in train.columns if x not in del_col]
features

['seg',
 'Hour',
 'Dayofweek',
 'IsWeekend',
 'IsHoliday',
 'Hour_sin',
 'Hour_cos',
 'longitude',
 'latitude',
 'Accident',
 'suburb',
 'length_1',
 'roadno',
 'surftype']

In [65]:
## Reduce training Features
val_train[features] = reduce_mem_usage(val_train[features] )
val_test[features] = reduce_mem_usage(val_test[features] )

Memory usage of dataframe is 373.16 MB


/home/zindistars2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/zindistars2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/zindistars2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

Memory usage after optimization is: 120.24 MB
Decreased by 67.8%
Memory usage of dataframe is 103.10 MB
Memory usage after optimization is: 33.22 MB
Decreased by 67.8%


/home/zindistars2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/zindistars2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/zindistars2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

In [66]:
## lgb Parametres

lgb_params = {
          "booster": "gbtree",
          "eta": 0.05,
         'num_iterations': 190,
         'verbosity': 50,
         'silent': 1,
         'eval_metric': 'binary_logloss',
         'seed': 2020,
         'metric': 'binary_logloss',
         "max_depth": 15,
          'is_unbalance': True,
          "subsample": 0.8,
          "colsample_bytree": 0.8,
          "alpha": 0.4
}



## Cat_boost Parametres
cat_params= {
            'num_boost_round': 250,
            'use_best_model':True,
            'depth': 8,
            'random_state': 2020,
            'eval_metric': 'AUC', 
            'loss_function': 'Logloss',
            'verbose': True
}
               

### Lgb

In [67]:
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import confusion_matrix, f1_score, precision_recall_curve
import matplotlib.pyplot as plt

In [68]:
trn_data = lgb.Dataset(val_train[features], label=val_train['target'])
clf = lgb.train(lgb_params, trn_data, 190)

predictions_lgb = clf.predict(test[features])
oof_lgb = clf.predict(val_train[features])
print ('OOf score', f1_score(val_train['target'], (oof_lgb > 0.05).astype(int)))

/home/zindistars2/anaconda3/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/zindistars2/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


OOf score 0.13497939684468013


In [69]:
## 0.13560956148808137

### CatBoost

In [70]:
import gc
import catboost
from catboost import CatBoostClassifier,Pool,cv,CatBoost

In [71]:
trn_data = Pool(val_train[features], label=val_train['target'],cat_features = cat_cols)

clf = CatBoost(cat_params)
clf.fit(trn_data,verbose_eval = True,eval_set=(trn_data))  
predictions_cat = clf.predict(test[features],prediction_type='Probability')[:,1]
oof_cat = clf.predict(val_train[features],prediction_type='Probability')[:,1]
print ('OOf score', f1_score(val_train['target'], (oof_cat > 0.05).astype(int)))

Learning rate set to 0.458621
0:	test: 0.7869819	best: 0.7869819 (0)	total: 4.06s	remaining: 16m 51s
1:	test: 0.8344682	best: 0.8344682 (1)	total: 10.3s	remaining: 21m 17s
2:	test: 0.8383004	best: 0.8383004 (2)	total: 16.5s	remaining: 22m 42s
3:	test: 0.8454366	best: 0.8454366 (3)	total: 20.3s	remaining: 20m 49s
4:	test: 0.8493965	best: 0.8493965 (4)	total: 23.7s	remaining: 19m 18s
5:	test: 0.8571752	best: 0.8571752 (5)	total: 26.9s	remaining: 18m 13s
6:	test: 0.8582134	best: 0.8582134 (6)	total: 30.4s	remaining: 17m 33s
7:	test: 0.8591718	best: 0.8591718 (7)	total: 33.7s	remaining: 16m 59s
8:	test: 0.8606640	best: 0.8606640 (8)	total: 37s	remaining: 16m 30s
9:	test: 0.8633771	best: 0.8633771 (9)	total: 40.1s	remaining: 16m 3s
10:	test: 0.8654979	best: 0.8654979 (10)	total: 43.7s	remaining: 15m 49s
11:	test: 0.8657193	best: 0.8657193 (11)	total: 47.1s	remaining: 15m 34s
12:	test: 0.8670872	best: 0.8670872 (12)	total: 50.3s	remaining: 15m 17s
13:	test: 0.8671776	best: 0.8671776 (13)	tot

112:	test: 0.8759059	best: 0.8759064 (110)	total: 6m 4s	remaining: 7m 22s
113:	test: 0.8759130	best: 0.8759130 (113)	total: 6m 7s	remaining: 7m 18s
114:	test: 0.8759088	best: 0.8759130 (113)	total: 6m 10s	remaining: 7m 15s
115:	test: 0.8759734	best: 0.8759734 (115)	total: 6m 14s	remaining: 7m 12s
116:	test: 0.8760216	best: 0.8760216 (116)	total: 6m 18s	remaining: 7m 9s
117:	test: 0.8760213	best: 0.8760216 (116)	total: 6m 21s	remaining: 7m 6s
118:	test: 0.8760227	best: 0.8760227 (118)	total: 6m 24s	remaining: 7m 3s
119:	test: 0.8760249	best: 0.8760249 (119)	total: 6m 28s	remaining: 7m
120:	test: 0.8760250	best: 0.8760250 (120)	total: 6m 30s	remaining: 6m 56s
121:	test: 0.8760244	best: 0.8760250 (120)	total: 6m 32s	remaining: 6m 52s
122:	test: 0.8760396	best: 0.8760396 (122)	total: 6m 36s	remaining: 6m 49s
123:	test: 0.8760404	best: 0.8760404 (123)	total: 6m 40s	remaining: 6m 46s
124:	test: 0.8760537	best: 0.8760537 (124)	total: 6m 43s	remaining: 6m 43s
125:	test: 0.8761147	best: 0.87611

222:	test: 0.8782709	best: 0.8782709 (222)	total: 11m 53s	remaining: 1m 26s
223:	test: 0.8782690	best: 0.8782709 (222)	total: 11m 55s	remaining: 1m 23s
224:	test: 0.8782704	best: 0.8782709 (222)	total: 11m 58s	remaining: 1m 19s
225:	test: 0.8782727	best: 0.8782727 (225)	total: 12m 2s	remaining: 1m 16s
226:	test: 0.8782770	best: 0.8782770 (226)	total: 12m 5s	remaining: 1m 13s
227:	test: 0.8782772	best: 0.8782772 (227)	total: 12m 8s	remaining: 1m 10s
228:	test: 0.8782784	best: 0.8782784 (228)	total: 12m 10s	remaining: 1m 7s
229:	test: 0.8782864	best: 0.8782864 (229)	total: 12m 14s	remaining: 1m 3s
230:	test: 0.8782904	best: 0.8782904 (230)	total: 12m 17s	remaining: 1m
231:	test: 0.8782953	best: 0.8782953 (231)	total: 12m 20s	remaining: 57.5s
232:	test: 0.8782988	best: 0.8782988 (232)	total: 12m 24s	remaining: 54.3s
233:	test: 0.8782818	best: 0.8782988 (232)	total: 12m 27s	remaining: 51.1s
234:	test: 0.8783737	best: 0.8783737 (234)	total: 12m 30s	remaining: 47.9s
235:	test: 0.8783930	best

In [72]:
## Aveage the  two models for train
print ('OOf score', f1_score(val_train['target'], ((0.5*oof_cat) + (0.5*oof_lgb) > 0.05).astype(int)))

OOf score 0.13776419242525784


In [73]:
## Average two models for test
predictions = (0.5*predictions_cat) + (0.5*predictions_lgb)

In [74]:
val_train['target'].value_counts(True)

0    0.995385
1    0.004615
Name: target, dtype: float64

## Evaluate the data

In [75]:
test['pred_prob'] = predictions
# test['prediction'] = (test['pred_prob']>0.07).astype(int) ## Ensembling_07        LB:0.123049956178791      
# test['prediction'] = (test['pred_prob']>0.06).astype(int) ## Ensembling_06        LB:0.12304120719675
# test['prediction'] = (test['pred_prob']>0.05).astype(int) ## Ensembling_05        LB:
test['prediction'] = (test['pred_prob']>0.065).astype(int) ## Ensembling_065       



test['prediction'].value_counts(True)

0    0.994913
1    0.005087
Name: prediction, dtype: float64

## Submission

In [76]:
test[['datetime x segment_id', 'prediction']].to_csv('Ensembling_065_v2.csv', index=False)

In [77]:
test.shape

(1174496, 45)

In [78]:
test.prediction.value_counts(True)

0    0.994913
1    0.005087
Name: prediction, dtype: float64